In [4]:
import numpy as np
import torch
from model.maddpg import MADDPG
from runEnv import UAVEnv
from env import load_different_scale_csv

device = "cuda" if torch.cuda.is_available() else "cpu"
# 设置不同的规模比例
min_scale = 5
middle_scale = 10
max_scale = 20

scale = min_scale
n_agents = scale
max_neighbors = scale / 3
# ---------- 加载环境 ----------
uavs, tasks, targets = load_different_scale_csv(
    "data/train/uav.csv",
    "data/train/task.csv",
    size = scale
)
env = UAVEnv(uavs, targets, tasks)

# 单个 UAV 观测维度（你 normalize 后的）
obs_dim = 9 * (max_neighbors + 1) + 5
total_obs_dim = obs_dim * scale

maddpg = MADDPG(
    n_agents=n_agents,
    obs_dim=obs_dim,
    total_obs_dim=total_obs_dim,
    device=device
)

TypeError: empty(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got float"

In [ ]:
# ---------- 训练 ----------
episodes = 500

for ep in range(episodes):
    state = env.reset()
    done = False
    ep_reward = 0

    while not done:
        # 构造 obs_all
        obs_all = env.get_obs_all()
        actions = maddpg.select_action(obs_all)
        chosen = np.argmax(actions)
        next_state, reward, done, _ = env.step(chosen)
        if next_state is None:
            break
        next_obs_all = env.get_obs_all()

        maddpg.buffer.push(
            obs_all,
            actions,
            reward,
            next_obs_all,
            done
        )
        maddpg.update()
        state = next_state
        ep_reward += reward

    print(f"Episode {ep}, Reward: {ep_reward:.2f}")